In [1]:
#get a list of genes and their canonical TSS and 

In [2]:
import pandas as pd
from bisect import bisect_left
data='../data/'

def fn(myList, myNumber): #utility function to get the closest posision from myList to a give posision myNumber
    pos = bisect_left(myList, myNumber)
    if pos == 0:
        return myList[0]
    if pos == len(myList):
        return myList[-1]
    before = myList[pos - 1]
    after = myList[pos]
    if after - myNumber < myNumber - before:
        return after
    else:
        return before 
def get_fn(man,genes,strands): #get name, distance, and direction of the closest gene (based on TSS)
    dis=[]
    name=[]
    strand=[]
    tss=[]
    for i,b in man[['g']].iterrows():
        fns=fn(genes.index,b['g'])
        tss.append(fns)
        dis.append(b['g']-fns)
        name.append(genes.loc[fns])     
        strand.append(strands.loc[fns])
    man['dis']=dis
    man['gene']=name
    man['strand']=strand
    man['tss']=tss
    return man

In [3]:
# get TSSs (for coding genes). biomart.txt from biomart website
gen=pd.read_table(data+'input/biomart.txt',nrows=10**6,index_col=0,
                 usecols=['Gene stable ID','Protein stable ID', 'Gene name', 'Strand', 'Transcript start (bp)',
       'Transcript end (bp)', 'Chromosome/scaffold name','Ensembl Canonical']) # from Ensemble gencode, limit:chr1-22 , features:ch, strand, prot, chr, beg, end, ENSG, cannonical, symbol, 
gen.columns=['prot','gen','+','beg','end','can','ch'] 
gen=gen[gen['prot'].notna()].copy() ### protein coding
gen['ts']=gen.apply(lambda x: x['beg'] if x['+']==1 else x['end'],axis=1)
gen=gen[['gen','ch','ts','+']].drop_duplicates()
gen['tss']=gen['ch'].astype(int)*10**9+gen['ts']
genes=gen[['gen','tss']].drop_duplicates().groupby('tss')['gen'].agg("first")
strands=gen[['+','tss']].drop_duplicates().groupby('tss')['+'].agg("first")
print(gen.shape)
gen.head(1)

(90830, 5)


,gen,ch,ts,+,tss
Gene stable ID,,,,,
ENSG00000142611,PRDM16,1,3069168,1,1003069168


In [4]:
gend=gen['gen'].drop_duplicates()
gend.to_csv(data+'/clean/genes.csv')
len(gend)

18937

In [5]:
%%time
#EPIC1 manifest from Illumina website, remove sex chromosomes, get global coordinates
man=pd.read_csv(data+'input/infinium-methylationepic-v-1-0-b5-manifest-file.csv',skiprows=7,index_col=0, 
                usecols=['IlmnID','CHR_hg38','Start_hg38','Forward_Sequence','UCSC_RefGene_Name','UCSC_RefGene_Group','Relation_to_UCSC_CpG_Island']) ##### MESA is EPIC v1
man=man[man['CHR_hg38'].isin(['chr'+str(c) for c in range(1,23)])].copy()
man['ch']=man['CHR_hg38'].str[3:].astype(int)
man['pos']=man['Start_hg38'].astype(int)
man['g']=man['ch'].astype(int)*10**9+man['pos']
man_out=get_fn(man,genes,strands)
man_out.to_csv(data+'clean/man_e1.csv')

CPU times: user 49.1 s, sys: 1.11 s, total: 50.2 s
Wall time: 51 s


In [ ]:
%%time
#EPIC2 manifest from Illumina website, remove sex chromosomes, get global coordinates 
man=pd.read_csv(data+'input/EPIC-8v2-0_A1.csv',skiprows=7,index_col=0, low_memory=False,
                usecols=['IlmnID','Name','CHR','MAPINFO','Forward_Sequence','UCSC_RefGene_Name','UCSC_RefGene_Group','Relation_to_UCSC_CpG_Island']) ##### MGB is EPIC v2 A1
man=man[man['CHR'].isin(['chr'+str(c) for c in range(1,23)])].copy()
man['ch']=man['CHR'].str[3:].astype(int)
man['pos']=man['MAPINFO'].astype(int)
man['g']=man['ch'].astype(int)*10**9+man['pos']
man_out=get_fn(man,genes,strands)
man_out.to_csv(data+'clean/man_e2.csv')